<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Алексей, привет! Спасибо, что прислал работу во время :) 
    
Меня зовут Евгений, и я буду проверять твой проект) 
Ты проделал большую работу над проектом, однако в любом деле всегда можно можно что-то улучшить. Моя основная цель — не показать, что где—то совершены какие—либо ошибки, а поделиться своим опытом, который поможет тебе в дальнейших работах. Далее в файле ты сможешь увидить мои комментарии, постарайся учесть их при выполнении следующих заданий.    

Мои комментарии обозначены пометкой **Комментарий ревьюера**. Далее в файле ты сможешь найти их в похожих ячейках (если фон комментария <font color=green>зелёный</font> - всё сделано правильно; <font color=orange>оранжевый</font> - есть замечания, но не критично; <font color=red>красный</font> - нужно переделать). Не удаляй эти комментарии и постарайся учесть их в ходе выполнения данного проекта. 

</div>

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div class="alert alert-warning">
<h2> Комментарий ревьюера</h2>

Рекомендуется делать краткое вступление в работу и описание того, что планируешь сделать (если план у тебя уже сформирован), а также информацию о входных данных: какие столбцы есть в таблице, их названия и какую информацию они несут. Так работа станет выглядеть презентабельнее.

</div>

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd 

df = pd.read_csv('/datasets/data.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Будем рассматривать столбцы по очереди и искать аномалии.
Начнем со стоблца `children`, в нем видно, что есть большое (20) и отрицательное (-1) значения 

In [2]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Посмотрев на столбец `days_employed` можно увидеть очень большие, отрицательные и "NaN" значения.
Также этот столбец имеет тип `float64`

In [3]:
print(df['days_employed'].isnull().sum())
print(df['days_employed'].max())
print(df['days_employed'].min())

2174
401755.40047533
-18388.949900568383


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Отлично, что проверяем стандартные статистические метрики. У нас есть готовый инструмент, который откроет сразу все стандартные характеристики — `describe()`.

</div>

В столбце `education` можно увидеть, что есть много одинаковых значений с разным регистром

In [4]:
print(df['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


В столбце `gender` есть одно лишнее значение

In [5]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Столбец `debt` имеет тип `int64`

In [6]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

В столбце `total_income` есть пропущенные значения

In [7]:
df['total_income'].isnull().sum()

2174

### Вывод

Проблемы найденные в датасете:
* Большое и отрицательное значение в столбце `children`
* Пропущенные данные в столбце `days_employed`
* Столбец `days_employed` имеет тип `float64` вместо `int64`
* Столбец `days_employed` иммет очень большие и отрицателдьные значения
* В столбце `education` одинаковые строки с разными регистрами
* Столбец `debt` иммет тип `int64`
* Пропущенные данные с толбце `total_income`

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Хорошо «сырые» данные посмотрели, теперь давай их обработаем.
    
</div>

### Шаг 2. Предобработка данных

### Обработка пропусков

Сначала разберемся с пропусками в `total_income`.  
Для этого найдем медианные зарплаты для работающих и неработающих людей.  
Работающие люди - это те, у кого в `income_type` стоят "сотрудник","компаньон", "госслужащий" или "предприниматель".  
Неработающие - все остальные.

In [8]:
working_income_types = ['сотрудник','компаньон', 'госслужащий', 'предприниматель']
non_working_income_types = ['пенсионер', 'безработный', 'в декрете', 'студент']

median_working_income = df[df['income_type'].isin(working_income_types)]['total_income'].median()
median_non_working_days_employed = df[df['income_type'].isin(non_working_income_types)]['total_income'].median()

print(median_working_income)
print(median_non_working_days_employed)

151147.3268776098
118480.8374078531


Теперь поставим это значение в пустых строках `total_income`

In [9]:
df.loc[df['total_income'].isnull() & df['income_type'].isin(working_income_types), 'total_income'] = median_working_income
df.loc[df['total_income'].isnull() & df['income_type'].isin(non_working_income_types), 'total_income'] = median_non_working_days_employed

print(df['total_income'].isnull().sum())

0



<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Очень хорошо, что производим замену пропущенных значений с помощью выявленных групп.  
 
Стоит отметить, что люди с разными профессиями получают разный доход, соответственно будет здорово детальнее применить обработку по группам. 
    
Давай попробуем ещё немного упростить с помощью методов `groupby()`, `trasform()` и `fillna()`.
    
Руководствоваться будем следующей логикой:
    
1. Создадим отдельный столбец, в который поместим медианное значение для каждого объекта исходя из группы профессий. В этом нам поможет метод `transform()`
2. Далее заменим пропуски используя полученный столбец
    
</div>

Теперь повторим эти шаги для `days_employed`  
Найдем медианные значения для работающих и неработающих людей 

In [10]:
median_working_days_employed = df[df['income_type'].isin(working_income_types)]['days_employed'].median()
median_non_working_days_employed = df[df['income_type'].isin(non_working_income_types)]['days_employed'].median()

print(median_working_days_employed)
print(median_non_working_days_employed)

-1630.0193809778216
365139.36728456186


Видно, что значение `days_employed` для работающих людей меньше 0.  
А значение для безработных людей больше 0 и очень большое (365139 / 360 = 1014 лет)

Причина этого видимо в том, что для работающих людей указывается время работы в днях с отрицательным значением, а для неработающих людей время работы с положительным значением в часах (365139 / 24 / 360 = 42 года)
Для текущей задачи нам эти значения не понадобятся, поэтому просто присвоим пустым значениям `days_employed` соответствующие медианные значения 


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Хорошо, что подумал о причинах ошибок в данных. Какие рекомендации можно дать в смежные отделы компании  ?    
</div>

In [11]:
df.loc[df['days_employed'].isnull() & df['income_type'].isin(working_income_types), 'days_employed'] = median_working_days_employed
df.loc[df['days_employed'].isnull() & df['income_type'].isin(non_working_income_types), 'days_employed'] = median_non_working_days_employed

print(df['days_employed'].isnull().sum())

0


Проверим, что больше не осталось столбцов с пропусками

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Пропущенные значения в `total_income` и `days_employed` были заменены на медианные значения для соответствующих значений `income_type`

### Замена типа данных

Из вывода функции `info()` из шага 1 видно, что значение в столбце `days_employed` имеет значение `float64`. 
Его можно заменить на значение `int64` при помощи метода `astype()`, поскольку нам не нужная такая точность для этого столбца

In [13]:
df['days_employed'] = df['days_employed'].astype('int64')

Тип столбца `debt` можно зменить на `bool`, посмольку значения в нем выполняют логическую функцию

In [14]:
df['debt'] = df['debt'].astype('bool')

Проверим, что типы столбцов были изменены верно

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null bool
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: bool(1), float64(1), int64(5), object(5)
memory usage: 1.8+ MB


### Вывод

Тип столбца `days_employed` был заменен на `int64`, а стобца `debt` на `bool` для того, чтобы лучше отражать тип хранимых данных.


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Как думаешь, для чего мы заменяем типы данных?     
</div>

### Обработка дубликатов

Если посмотреть на значения в столбце `education`, то можно увидеть, что там есть одинаковые значения, но с разными регистрами

In [16]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Приведем строки к нижнему регистру

In [17]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

### Вывод

К столбцу `education` был применен метод `lower()` для того, чтобы привести все строки к одному регистру, поскольку в нем были строки с одинаковыми значениями, но разными регистрами.  
Скорее всего такая ситуация произошла из-за того, что образование операторы забивали руками 

### Лемматизация

Импортируем бибилиотеку `pymystem3` и посчитаем самые повторяющиеся леммы среди целей получения кредита

In [18]:
from pymystem3 import Mystem
from collections import Counter
import itertools

m = Mystem() 

lemmas = [m.lemmatize(x) for x in df['purpose'].tolist()]
print(Counter(itertools.chain.from_iterable(lemmas)))

Counter({' ': 33677, '\n': 21525, 'недвижимость': 6367, 'покупка': 5912, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'проведение': 777, 'сыграть': 774, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 612, 'подержанный': 489, 'подержать': 479, 'приобретение': 462, 'профильный': 436})


### Вывод

Леммитизировали все цели получения кредита и посчитали количество лемм

### Категоризация данных

Проанализировав леммы из предыдущего задания можно построить следующий словарь из леммы в категорию займа

In [19]:
lemma_to_purpose = {
    'недвижимость': 'real_estate',
    'жилье': 'real_estate',
    'автомобиль': 'car', 
    'образование': 'education', 
    'свадьба': 'wedding',
}

Заведем новый столбец `purpose_group` и применим функцию `purpose_group()`, которая вычислит группу для цели получения кредита.
После этого проверим, что пустых значений в `purpose_group` нет 

In [20]:
def purpose_group(purpose):
    lemmas = m.lemmatize(purpose)
    for lemma in lemmas:
        group = lemma_to_purpose.get(lemma)
        if group:
            return group
        
df['purpose_group'] = df['purpose'].apply(purpose_group)
print(df['purpose_group'].isnull().sum())
display(df.head(5))

0


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875.639453,покупка жилья,real_estate
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080.014102,приобретение автомобиля,car
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885.952297,покупка жилья,real_estate
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628.550329,дополнительное образование,education
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616.077870,сыграть свадьбу,wedding


Добавим еще один столбец `has_children`, в котором поставим "True", если у человека есть хотя бы один ребенок или "False" в противном случае.

In [21]:
def has_children(children):
    if children > 0: 
        return True
    return False
        
df['has_children'] = df['children'].apply(has_children)
display(df.head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,has_children
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875.639453,покупка жилья,real_estate,True
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080.014102,приобретение автомобиля,car,True
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885.952297,покупка жилья,real_estate,False
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628.550329,дополнительное образование,education,True
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616.077870,сыграть свадьбу,wedding,False


Также добавим столбец `total_income_group`, в котором выделим 3 группы: назкий, средний и выский доход 

In [22]:
def total_income_group(total_income):
    if total_income <= df['total_income'].quantile(0.33):
        return 'low'
    if total_income <= df['total_income'].quantile(0.66):
        return 'medium'
    return 'high'

df['total_income_group'] = df['total_income'].apply(total_income_group)
display(df.head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,has_children,total_income_group
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875.639453,покупка жилья,real_estate,True,high
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080.014102,приобретение автомобиля,car,True,low
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885.952297,покупка жилья,real_estate,False,medium
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628.550329,дополнительное образование,education,True,high
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616.077870,сыграть свадьбу,wedding,False,medium



<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Отлично применять стандартные статистические решения для определения уровней дохода.
    
---
    
Есть один нюанс, который мешает функции работать быстрее. Мы каждый раз, для каждой ячейки определяем квантиль 0.33 и 0.66, а это увеличивает срок обработки. 
    
Для того, что проверить сколько выходит по времени мы можем добавить магическую команду `%%time`, которая покажет нам время которое мы потратили на обработку, будет ориентировочно 25—40 сек., достаточно долго для такого маленького датасета.
    
Сейчас наши данные исчисляются 20к строк и обрабатываются за 1 минуту отриентировочно, легко можно высчитать математически — сколько они будут обрабатываться при 100 млн. строк, достаточно долго :)
    
Поэтому важно помнить оптимизацию кода.
    
Как решить эту проблему ? Например определить квантили заранее и ссылать на них в нашей функции.
    

    
</div>

Предобработка данных завершена:

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 15 columns):
children              21525 non-null int64
days_employed         21525 non-null int64
dob_years             21525 non-null int64
education             21525 non-null object
education_id          21525 non-null int64
family_status         21525 non-null object
family_status_id      21525 non-null int64
gender                21525 non-null object
income_type           21525 non-null object
debt                  21525 non-null bool
total_income          21525 non-null float64
purpose               21525 non-null object
purpose_group         21525 non-null object
has_children          21525 non-null bool
total_income_group    21525 non-null object
dtypes: bool(2), float64(1), int64(5), object(7)
memory usage: 2.2+ MB


### Вывод

Был добавлен новый столбец `purpose_group`, в который сгруппированы на основе лемм типы получения кредита.   
Также был добавлен столбец `has_children` в котором было записано есть ли у человека дети

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Выведем сводную таблицу по `has_children` и `debt`.  
Добавим в нее строку `sum`, которая отображает сумму всех возвратов и невозвратов для людей с детьми и без.  
Таже вычислим сторку `debt_percent`, которая обозначает соотношение невозратов к общему кол-ву взятых кредитов.

In [24]:
has_children_debt = df.pivot_table(index='has_children', columns='debt', values='children', aggfunc='count')
has_children_debt['sum'] = has_children_debt[0] + has_children_debt[1]
has_children_debt['debt_percent'] = has_children_debt[1] / has_children_debt['sum']
display(has_children_debt)

debt,False,True,sum,debt_percent
has_children,,,,
False,13132,1064,14196,0.074951
True,6652,677,7329,0.092373


### Вывод

Из таблицы можно сделать вывод, что люди с детьми чаще не возвращают кредиты.


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно.   
</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Выведем сводную таблицу по столбцам `family_status` и `debt` и отсортируем ее по убыванию

In [25]:
family_status_debt = df.pivot_table(index='family_status', columns='debt', values='children', aggfunc='count')
family_status_debt['sum'] = family_status_debt[0] + family_status_debt[1]
family_status_debt['debt_percent'] = 100 * family_status_debt[1] / family_status_debt['sum'] 
display(family_status_debt.sort_values('debt_percent', ascending=False))

debt,False,True,sum,debt_percent
family_status,,,,
Не женат / не замужем,2539,274,2813,9.740491
гражданский брак,3789,388,4177,9.288963
женат / замужем,11449,931,12380,7.520194
в разводе,1110,85,1195,7.112971
вдовец / вдова,897,63,960,6.562500


### Вывод

Из представленной таблицы можно сделать вывод, что чаще кредит не отдают неженатые люди, а реже всего - вдовцы.  
Но чаще всего кредит берут женатые люди, а реже всего - вдовцы.


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно.   
    
---
    
Обрати пожалуйста внимание, что мы используем повторяющиеся методы в ячейках 24—27. Это означает только одно — оптимизация кода. 

То есть здесь напрашивается функция, которая будет определена один раз и применена необходимое количество раз.
    
</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Выведем сводную таблицу по столбцам `total_income_group` и `debt` и после этого отсортируем ее по убыванию по кол-ву невозвратов. 

In [26]:
purpose_debt = df.pivot_table(index='total_income_group', columns='debt', values='children', aggfunc='count')
purpose_debt['sum'] = purpose_debt[0] + purpose_debt[1]
purpose_debt['debt_percent'] = 100 * purpose_debt[1] / purpose_debt['sum'] 
display(purpose_debt.sort_values('debt_percent', ascending=False))

debt,False,True,sum,debt_percent
total_income_group,,,,
medium,6481,606,7087,8.550868
low,6540,579,7119,8.133165
high,6763,556,7319,7.596666


### Вывод

Из таблицы можно сделать вывод, что люди со всредним доходом отдают кредиты реже всего, а люди с высоким - чаще всего.


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно.   
</div>

- Как разные цели кредита влияют на его возврат в срок?

Выведем сводную таблицу по столбцам `purpose_group` и `debt` и после этого отсортируем ее по убыванию по кол-ву невозвратов. 

In [27]:
purpose_debt = df.pivot_table(index='purpose_group', columns='debt', values='children', aggfunc='count')
purpose_debt['sum'] = purpose_debt[0] + purpose_debt[1]
purpose_debt['debt_percent'] = 100 * purpose_debt[1] / purpose_debt['sum'] 
display(purpose_debt.sort_values('debt_percent', ascending=False))

debt,False,True,sum,debt_percent
purpose_group,,,,
car,3912,403,4315,9.339513
education,3652,370,4022,9.199403
wedding,2162,186,2348,7.921635
real_estate,10058,782,10840,7.214022


### Вывод

Из таблицы можно сделать вывод, что чаще всего люди не отдают кредит на машину, а реже всего на недвижимость


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно.   
</div>

### Шаг 4. Общий вывод

В данном задании был проведен анализ надежности заемщиков.  
Мы вывели несколько характеристик надежных заемщиков:
* высокий доход  
* отсутсвие детей 

Таже мы сделали вывод, что люди чаще возвращают кредиты, взятые на покупку недвижимости.

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Выводы понятны и логичны. Рекомендую подкреплять его выявленными фактами. Так он будет более конкретным.
    
Обрати пожалуйста внимание, что финальный вывод и есть главный результат твоей работы. Стоит писать его подробно по результатам проведенной работы. Также можно расписать все, что было сделано в работе.

Хотел бы ещё раз коснуться вопроса оптимизации. Всё приходит с опытом, поэтому не стоит переживать, что было сделано что—то не так. Мы учимся и это нормально :)
    
В целом сам проект выполнен на хорошем уровне! На протяжении всей работы чувствуется глубина проработки задачи и это безусловный плюс для нас. Было использовано большое количество методов, которые будут помогать тебе в дальнейших проектах.
    
Не стоит забывать про оформление своей работы, это важный такой же важный этап. Рекомендую «не стесняться» использовать ячейки типа **markdown** там где нам необходимо отобразить свои размышления, в том числе с использованием дополнительной стилизации.
  
Успехов тебе в новых проектах!
    
</div>

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Рекомендую посмотреть на дополнительную стилизацию ячеек типа **markdown**.
    
**Жирный**
*Курсив*
    
--- 

# Заголовок первого уровня

--- 
    
## Заголовок первого уровня
  
--- 
Списки:
    
- один 
- два
- три
   
---
    
1. Раз
2. Раз раз
3. Раз два три
    
--- 
    
Отображение `переменных`
    
    
</div>

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.